##Data Kemdikbud scraper
Because somehow they make it very painful to just download the data.

In [1]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd

C:\Anaconda2\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


Get list of kabupaten/kota

In [21]:
master = urllib.urlopen("http://sekolah.data.kemdikbud.go.id/index.php/cpetasebaran/index/000000/")
mastersoup = BeautifulSoup(master)

In [22]:
list_daerah=[]
for opt in mastersoup.find_all('option'):
    daerah={}
    sc_code = opt.get('value')
    if sc_code:
        daerah["code"]=sc_code
        daerah["name"]=''.join(opt.findAll(text=True))
        list_daerah.append(daerah)
list_daerah = list_daerah[:-8]
list_daerah

[{'code': '060000', 'name': u'Prop. Aceh'},
 {'code': '220000', 'name': u'Prop. Bali'},
 {'code': '290000', 'name': u'Prop. Bangka Belitung'},
 {'code': '280000', 'name': u'Prop. Banten'},
 {'code': '260000', 'name': u'Prop. Bengkulu'},
 {'code': '040000', 'name': u'Prop. D.I. Yogyakarta'},
 {'code': '010000', 'name': u'Prop. D.K.I. Jakarta'},
 {'code': '300000', 'name': u'Prop. Gorontalo'},
 {'code': '100000', 'name': u'Prop. Jambi'},
 {'code': '020000', 'name': u'Prop. Jawa Barat'},
 {'code': '030000', 'name': u'Prop. Jawa Tengah'},
 {'code': '050000', 'name': u'Prop. Jawa Timur'},
 {'code': '130000', 'name': u'Prop. Kalimantan Barat'},
 {'code': '150000', 'name': u'Prop. Kalimantan Selatan'},
 {'code': '140000', 'name': u'Prop. Kalimantan Tengah'},
 {'code': '160000', 'name': u'Prop. Kalimantan Timur'},
 {'code': '340000', 'name': u'Prop. Kalimantan Utara'},
 {'code': '310000', 'name': u'Prop. Kepulauan Riau'},
 {'code': '120000', 'name': u'Prop. Lampung'},
 {'code': '210000', 'name

Scrape marker in the response

In [43]:
#test: jakarta, SD
f = urllib.urlopen("http://sekolah.data.kemdikbud.go.id/index.php/cpetasebaran/index/010000/SD") 
soup = BeautifulSoup(f)
sc = soup.find_all('script')[21].string

In [44]:
sc_c = sc.strip(' \t\n\r').split(";")
sc_arr = []
for s in sc_c:
    sc_arr.append(s.strip(' \t\n\r'))

In [45]:
sc_name=[]
sc_latlong=[]
for line in sc_arr:
    school={}
    if "NPSN" in line:
        #url
        school["url"] = line.split("=")[7][25:61]
        #NPSN
        school["npsn"]= line.split("=")[5][36:44]
        #nama
        sc_str = line.split("=")[8]
        school["name"] = sc_str[9:sc_str.find("</a")] 
        sc_name.append(school)   
    if "latLng" in line:
        latlng = line.split("ng(")[1][0:20].split(",")
        school["lat"] = latlng[0]
        school["long"] = latlng[1]
        sc_latlong.append(school)
#merge two dict list
for n, l in zip(sc_name,sc_latlong):
    n.update(l)

load into pandas dataframe

In [29]:
d = pd.DataFrame.from_dict(sc_name)

In [33]:
d.head()

,lat,long,name,npsn,url
0,-6.2582000,106.77620,SDN Kebayoran Lama Selatan 09 Pagi,20105964,809DE594-2BF5-E011-8E05-672CC87BAF72
1,-6.1650000,106.72820,SD NEGERI RAWA BUAYA 03 PG,20105474,E0CCD094-2BF5-E011-BE6F-D5744B4E5D2A
2,-6.2465000,106.76800,SD NEGERI ULUJAMI 01 PAGI,20102196,20BCE294-2BF5-E011-8A6A-E1606F111B44
3,-6.2408000,106.88140,SD NEGERI CIPINANG CEMPEDAK 04,20108523,C0F40595-2BF5-E011-98BF-23D5A0CA8166
4,-6.2362000,106.77230,SDN Grogol Utara 15 Petang.,20105918,8041E594-2BF5-E011-848B-D7583B02BDF0


In [51]:
d['level'] = "SD"
#d_smp['level'] = "SMP"
#d_sma['level'] = "SMA"
#d_smk['level'] = "SMK"
#d_all = pd.concat([d,d_smp,d_sma,d_smk])
d.head()

,lat,long,name,npsn,url,level
0,-6.2582000,106.77620,SDN Kebayoran Lama Selatan 09 Pagi,20105964,809DE594-2BF5-E011-8E05-672CC87BAF72,SD
1,-6.1650000,106.72820,SD NEGERI RAWA BUAYA 03 PG,20105474,E0CCD094-2BF5-E011-BE6F-D5744B4E5D2A,SD
2,-6.2465000,106.76800,SD NEGERI ULUJAMI 01 PAGI,20102196,20BCE294-2BF5-E011-8A6A-E1606F111B44,SD
3,-6.2408000,106.88140,SD NEGERI CIPINANG CEMPEDAK 04,20108523,C0F40595-2BF5-E011-98BF-23D5A0CA8166,SD
4,-6.2362000,106.77230,SDN Grogol Utara 15 Petang.,20105918,8041E594-2BF5-E011-848B-D7583B02BDF0,SD


In [52]:
#assuming separate dataframe are made for each level
d_all = pd.concat([d,d_smp,d_sma,d_smk])

In [53]:
d_all.to_csv("jakarta.csv")